In [1]:
import pandas as pd
import numpy as np
import math

In [35]:
ASSETS_DIR          = f"./assets"
GDP_DATASET         = f"{ASSETS_DIR}/PIB_por_ano.csv"
POPULATION_DATASET  = f"{ASSETS_DIR}/estimativa_populacao-v2.csv"
SALARY_DATASET      = f"{ASSETS_DIR}/massa_salarial_por_ano.csv"
UC_PER_YEAR_DATASET = f"{ASSETS_DIR}/UCxAno.csv"
EMPLOYMENT_DATASET  = f"{ASSETS_DIR}/total_de_empregados_por_ano.csv"
# TOTAL_EMPRESAS_DATASET_DIR = f"{ASSETS_DIR}"

In [54]:
gdp_df = pd.read_csv(GDP_DATASET)
population_df = pd.read_csv(POPULATION_DATASET)
salary_df = pd.read_csv(SALARY_DATASET)
employee_df = pd.read_csv(EMPLOYMENT_DATASET)
uc_df = pd.read_csv(UC_PER_YEAR_DATASET)



In [65]:
relation_df = pd.DataFrame()

employees = []
salaries = []
population = []
uc_count = []

for year, count in zip(uc_df["Ano"], uc_df['Quantidade de UC']):
    try:
        key = str(year)

        sal = salary_df[key][0]
        pop = population_df[key][0]
        empl = employee_df[key][0]

    except KeyError:
        continue

    salaries.append(sal)
    population.append(pop)
    employees.append(empl)
    uc_count.append(count)

relation_df = pd.DataFrame.from_dict({
    'UCs': uc_count,
    'Empregados': employees,
    'Populacao': population,
    'Massa Salarial': salaries,
})

relation_df

# for year, count in zip(uc_df["Ano"], uc_df["Quantidade de UC"]):
#     try:
#         key = str(year)
#         print(year, count, , salary_df[key][0], population_df[key][0])
#     except KeyError:
#         continue
# print(employees)

,UCs,Empregados,Populacao,Massa Salarial
0,2201,1814187,7075494.0,4.132690e+09
1,2376,1875109,17677137.0,4.301976e+09
2,2468,1912155,17830257.0,4.416118e+09
3,2546,2060145,7338473.0,5.379025e+09
